<h2>XOR solution using Neural Network over relational graph</h2>

The exclusive OR (XOR) is a logical operation. The output of this operator is only true when the inputs are different from each other. 

<table>
  <tr>
    <th>A</th>
    <th>B</th>
    <th>Output</th>
  </tr>
    <td>0</td>
    <td>0</td>
    <td>0</td>
  </tr>
  </tr>
    <td>0</td>
    <td>1</td>
    <td>0</td>
  </tr>
  </tr>
    <td>1</td>
    <td>0</td>
    <td>0</td>
  </tr>
  </tr>
    <td>1</td>
    <td>1</td>
    <td>0</td>
  </tr>
</table>

The XOR problem is classic in neural networks. This problem was mentioned in the 1969 book "Perceptrons" by Marvin Minsky and then generated many debates around it. 

The image below represents OR and XOR problem. Note that for OR problem, you can simply draw a straight line to classify the outputs. But you can't do the same for XOR problem. There is not a linear function for this particular classification problem. As a solution, two lines or curves must be used to solve the classification. 

<img src="https://drive.google.com/uc?id=12xdVWz4PSQNCFA3RMQAJtppzZKFOOirm" width="600"><br />
<small>Image extracted from https://dev.to/jbahire/demystifying-the-xor-problem-1blk</small>


In [ ]:
from math import exp

'''
    The main idea is to replicate the representation from the paper Graph
    Structure of Neural Networks (https://arxiv.org/abs/2007.06559), using
    the approach of computing rounds of message exchange:
    Node feature: x1, x2, x3
    Message: fi(xj) = Wij * xj
    Aggregation: agg(sum(·))
    Rounds: n
'''

'''
    Node Class

    Represent the node in a graph. Can perform the following operations:
    - add_adge(node, weight): add a weighted bidirected edge between two nodes
    - send_message(target_node, fun, params): send a message to the target node
      represented by a function
    - aggregate_messages (fun): aggregate all messages received from the neighbours
      using a function (eg: sigmoid)
    - set_bias (bias)

'''
class Node:
  def __init__(self, node, value):
    self.id = node
    self.value = value
    self.neighbours = {}
    self.messages = []
    self.bias = 0

  def add_edge(self, node, weight = 0):
    self.neighbours[node.id] = { 'node': node, 'weight': weight }
    node.neighbours[self.id] = { 'node': self, 'weight': weight }

  def set_bias(self, bias):
    self.bias = bias

  def set_weight(self, node, weight):
    self.neighbours[node.id]['weight'] = weight

  def send_message(self, target_node, fun, params):
    message = fun(params, self.value)
    target_node.messages.append(message)
    return message

  def aggregate_messages(self, fun):
    messages = sum(self.messages) + self.bias
    updated_value = fun(messages)
    self.value = updated_value
    self.messages = []
    return updated_value

'''
    Graph Class
    
    Build a graph
    - add_node (node, value)
    - propagate (aggregation_fun, activation_fun): explore all nodes and propagate
      the message across its neighbours. It also perform the aggregation of all 
      messages received from the neighbours.
'''
class Graph:
  def __init__(self):
    self.nodes = {}
    self.num_nodes = 0
  
  def add_node(self, node, value):
    self.num_nodes = self.num_nodes + 1
    new_node = Node(node, value)
    self.nodes[node] = new_node
    return new_node

  def propagate(self, aggregation_fun, activation_fun):
    # send message to all neighbours
    for k, v in self.nodes.items():
      for _, n in v.neighbours.items():
        # print('Send message from', v.id, 'to', n['node'].id, ', weight', n['weight'])
        v.send_message(n['node'], aggregation_fun, n['weight'])
    # aggregate messages from all neighbours
    for k, v in self.nodes.items():
      v.aggregate_messages(activation_fun)

'''
    Class RelationalGraphNN

    Create a neural network using relational graph approach. Initialize
    the object with:
    - graph (Graph)
    - activation_function (function)
    - aggregation_function (function)
'''
class RelationalGraphNN:
  def __init__(self, graph):
    self.graph = graph
    self.round = None
    self.activation_function = None
    self.aggregation_function = None

  def run(self):
    for w in self.round[0]:
      source = self.graph.nodes[w['source']]
      target = self.graph.nodes[w['target']]
      source.set_weight(target, w['weight'])
    for b in self.round[1]:
      node = self.graph.nodes[b['node']]
      node.set_bias(b['bias'])
    self.graph.propagate(self.aggregation_function, self.activation_function)

  

In [ ]:
'''
    Implementation of relational NN to solve OR
'''
def solve_or(a, b):
  # Build a Graph with 2 nodes connected (undirected), and with self-loops
  graph = Graph()
  node_a = graph.add_node('a', value = a)
  node_b = graph.add_node('b', value = b)
  node_a.add_edge(node_b)
  node_b.add_edge(node_b)

  # Build a Relational Graph for the NN using the Graph created
  # - aggregation function is weight x feature
  # - activation function is sigmoid
  nn = RelationalGraphNN(graph)
  nn.aggregation_function = lambda w,x : w * x
  nn.activation_function = lambda x : 1.0 / (1.0 + exp(-x))

  # first round of message exchange
  # trainable weights and bias are defined manually
  exchange_round = [
  [{
    'source': 'a',
    'target': 'b',
    'weight': 20,
  },
  {
    'source': 'b',
    'target': 'b',
    'weight': 20
  }],
  [{
    'node': 'b',
    'bias': -10
  },
  ]]

  nn.round = exchange_round
  nn.run()

  # final output is in b node as convention
  output = nn.graph.nodes['b'].value

  return round(output,1)


In [ ]:
'''
    Implementation of relational NN to solve XOR.
    As XOR is non-linear separable, we need two rounds
    of message exchange.
'''
def solve_xor(a, b):
  # Build a Graph with 2 nodes connected (undirected), and with self-loops
  graph = Graph()
  node_a = graph.add_node('a', value = a)
  node_b = graph.add_node('b', value = b)
  node_a.add_edge(node_a)
  node_a.add_edge(node_b)
  node_b.add_edge(node_b)

  # Build a Relational Graph for the NN using the Graph created
  # - aggregation function is weight x feature
  # - activation function is sigmoid
  nn = RelationalGraphNN(graph)
  nn.aggregation_function = lambda w,x : w * x
  nn.activation_function = lambda x : 1.0 / (1.0 + exp(-x))

  # first round of message exchange
  # trainable weights and bias are defined manually
  exchange_round = [
  [{
    'source': 'a',
    'target': 'a',
    'weight': 20,
  },
  {
    'source': 'a',
    'target': 'b',
    'weight': -20,
  },
  {
    'source': 'b',
    'target': 'b',
    'weight': -20
  },
  {
    'source': 'b',
    'target': 'a',
    'weight': 20
  }],
  [
  {
    'node': 'a',
    'bias': -10
  },
  {
    'node': 'b',
    'bias': 30
  },
  ]]

  nn.round = exchange_round
  nn.run()

  # second round of message exchange
  # trainable weights and bias are defined manually
  exchange_round = [
  [{
    'source': 'a',
    'target': 'b',
    'weight': 20,
  },
  {
    'source': 'b',
    'target': 'b',
    'weight': 20
  }
  ],
  [{
    'node': 'b',
    'bias': -30
  }]
  ]

  nn.round = exchange_round
  nn.run()

  # final output is in b node as convention
  output = nn.graph.nodes['b'].value

  return round(output,1)


In [ ]:
# Test all ORs inputs
assert solve_or(0,0) == 0, "should be 0"
assert solve_or(0,1) == 1, "should be 1"
assert solve_or(1,0) == 1, "should be 1"
assert solve_or(1,1) == 1, "should be 1"
print("All tests passed for OR Neural Network")

All tests passed for OR Neural Network


In [ ]:
# Test all XORs inputs
assert solve_xor(0,0) == 0, "should be 0"
assert solve_xor(0,1) == 1, "should be 1"
assert solve_xor(1,0) == 1, "should be 1"
assert solve_xor(1,1) == 0, "should be 0"
print("All tests passed for XOR Neural Network")


All tests passed for XOR Neural Network
